In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# -*- coding: utf-8 -*-
"""
Created on Wed Oct  4 21:39:52 2017

@author: bhavesh
"""
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb

cwd=os.getcwd()+'/'

def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

df_train = pd.read_csv('input/train.csv')
df_test = pd.read_csv('input/test.csv')

target_train = df_train['target'].values
id_test = df_test['id'].values

df_train=df_train.drop(['target','id'],axis=1)
df_test=df_test.drop(['id'], axis = 1)
combine= pd.concat([df_train,df_test],axis=0)






  

/home/rodrigo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#print(combine['ps_car_13'].value_counts())

#https://stackoverflow.com/questions/10238357/finding-the-outlier-points-from-matplotlib-boxplot

#low = .05
#high = .95

#for f in combine.columns:
#    combine[f] = np.where(combine[f] == -1, combine[f].mean(), combine[f])
#    combine[f] = round((combine[f] - combine[f].mean()) / (combine[f].max() - combine[f].min()),1)
#    quant_df = pd.DataFrame([combine[f].quantile([low, high])])
#    max_out = quant_df[0.95].max()
#    combine[f] = np.where(combine[f] > max_out, max_out, combine[f])
#    print(combine[f].value_counts())

    

#combine['ps_car_13'] = round((combine['ps_car_13'] - combine['ps_car_13'].mean()) / (combine['ps_car_13'].max() - combine['ps_car_13'].min()),1)
#quant_df = pd.DataFrame([combine['ps_car_13'].quantile([low, high])])
#max_out = quant_df[0.95].max()
#combine['ps_car_13'] = np.where(combine['ps_car_13'] > max_out, max_out, combine['ps_car_13'])
#print(combine['ps_car_13'].value_counts())


#print(combine['ps_car_13'].value_counts())

#combine['ps_car_13_001'] = combine['ps_car_13'] * combine['ps_reg_03'] 
#combine['ps_car_13_002'] = combine['ps_car_13'] * combine['ps_car_12'] 
#combine['ps_car_13_003'] = combine['ps_car_13'] * combine['ps_reg_02'] 
#combine['ps_car_13_004'] = combine['ps_car_13'] * combine['ps_car_15'] 

#combine['ps_reg_03'] = round((combine['ps_reg_03'] - combine['ps_reg_03'].mean()) / (combine['ps_reg_03'].max() - combine['ps_reg_03'].min()),1)
#quant_df = pd.DataFrame([combine['ps_reg_03'].quantile([low, high])])
#max_out = quant_df[0.95].max()
#combine['ps_reg_03'] = np.where(combine['ps_reg_03'] > max_out, max_out, combine['ps_reg_03'])
#print(combine['ps_reg_03'].value_counts())


combine = combine.drop(['ps_reg_01','ps_reg_02','ps_reg_03','ps_car_12','ps_car_13','ps_car_15','ps_ind_14','ps_ind_12_bin'],1)


-0.3    469109
-0.1    359925
 0.0    205761
 0.4    153663
 0.2    127634
 0.6     87891
 0.3     84045
Name: ps_ind_01, dtype: int64
-0.1    1079327
 0.2     309747
 0.5      98431
-0.0        523
Name: ps_ind_02_cat, dtype: int64
 0.1    308030
-0.2    240652
-0.1    204836
-0.3    170922
-0.0    167980
 0.2    130452
 0.4    127750
 0.3     98528
-0.4     38878
Name: ps_ind_03, dtype: int64
-0.4    866864
 0.6    620936
-0.0       228
Name: ps_ind_04_cat, dtype: int64
-0.1    1319412
 0.6     101899
 0.4      20754
 0.1      20737
-0.0      14519
 0.3      10707
Name: ps_ind_05_cat, dtype: int64
-0.4    902572
 0.6    585456
Name: ps_ind_06_bin, dtype: int64
-0.3    1105415
 0.7     382613
Name: ps_ind_07_bin, dtype: int64
-0.2    1244343
 0.8     243685
Name: ps_ind_08_bin, dtype: int64
-0.2    1211754
 0.8     276274
Name: ps_ind_09_bin, dtype: int64
-0.0    1488028
Name: ps_ind_10_bin, dtype: int64
-0.0    1488028
Name: ps_ind_11_bin, dtype: int64
-0.0    1488028
Name: ps_ind_12

In [3]:
# Performing rounding/binding
#combine1= combine

float_features = [a for a in combine.columns if combine[a].dtypes =='float64']

for column in float_features:
    combine[column] = round(combine[column],1)
    
#combine.head()
#combine1.head()


In [4]:



# Performing one hot encoding
cat_features = [a for a in combine.columns if a.endswith('cat')]
for column in cat_features:
    temp=pd.get_dummies(pd.Series(combine[column]))
    combine=pd.concat([combine,temp],axis=1)
    combine=combine.drop([column],axis=1)

df_train=combine[:df_train.shape[0]]
df_test=combine[df_train.shape[0]:]

train = np.array(df_train)
test = np.array(df_test)

print ("The train shape is:",train.shape)
print ('The test shape is:',test.shape)


# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(train, target_train, test_size=0.2, random_state=42)

xgb_preds = []


#K = 5
#kf = KFold(n_splits = K, random_state = 3228,shuffle=True)


#for train_index, test_index in kf.split(train):
#    train_X, valid_X = train[train_index], train[test_index]
#    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
xgb_params = {'eta': 0.05, 'max_depth': 6, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

d_train = xgb.DMatrix(train_X, train_y)
d_valid = xgb.DMatrix(valid_X, valid_y)
d_test = xgb.DMatrix(test)
    
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model = xgb.train(xgb_params, d_train, 1000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=10, early_stopping_rounds=100)
                        
xgb_pred = model.predict(d_test)
xgb_preds.append(list(xgb_pred))
    
    
    
#preds=[]
#for i in range(len(xgb_preds[0])):
#    sum=0
#    for j in range(K):
#        sum+=xgb_preds[j][i]
#    preds.append(sum / K)

#output = pd.DataFrame({'id': id_test, 'target': preds})
#output.to_csv("{}-foldCV_avg_sub.csv".format(K), index=False)  

The train shape is: (595212, 117)
The test shape is: (892816, 117)
[0]	train-gini:0.218416	valid-gini:0.211909
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[10]	train-gini:0.248532	valid-gini:0.236761
[20]	train-gini:0.264547	valid-gini:0.246336
[30]	train-gini:0.267428	valid-gini:0.249023
[40]	train-gini:0.274874	valid-gini:0.252606
[50]	train-gini:0.282716	valid-gini:0.2563
[60]	train-gini:0.292422	valid-gini:0.260177
[70]	train-gini:0.304926	valid-gini:0.265963
[80]	train-gini:0.316459	valid-gini:0.270409
[90]	train-gini:0.329065	valid-gini:0.27302
[100]	train-gini:0.339779	valid-gini:0.276129
[110]	train-gini:0.349788	valid-gini:0.276635
[120]	train-gini:0.357767	valid-gini:0.278348
[130]	train-gini:0.365525	valid-gini:0.279186
[140]	train-gini:0.373991	valid-gini:0.279941
[150]	train-gini:0.382666	valid-gini:0.281112
[160]	train-gini:0.389953	valid-gini:0.28164
[170]	train-gini:0.3

In [5]:
#[10]  train-gini:0.250975	valid-gini:0.240113
#[10]  train-gini:0.252803	valid-gini:0.25038
#[10]  train-gini:0.254338	valid-gini:0.250544


#valid-gini:0.28738

#[594]	train-gini:0.438172	valid-gini:0.288515

trn_df = pd.read_csv("input/train.csv")

f_bins = [f for f in trn_df.columns if "_bin" in f]
f_cats = [f for f in trn_df.columns if "_cat" in f]
f_flts = [f for f in trn_df.columns if f not in f_bins + f_cats + ["target", "id"]]
f_all = [f_bins + f_cats + f_flts]



f_scores = pd.DataFrame(np.zeros((len(f_all), 2)), columns=["feature", "score"])
for i_f, f in enumerate(f_bins + f_cats + f_flts):
    # Get the score for each feature
    # If trn_df[f] has a negative score then - trn_df[f] has a positive one
    # Remember it's all about sorting !
    f_scores.loc[i_f] = [f, abs(gini_normalized(trn_df.target.values, trn_df[f].values))]
f_scores.sort_values(by="score", ascending=False, inplace=True)
f_scores




,feature,score
40,ps_car_13,0.153094
37,ps_reg_03,0.114447
39,ps_car_12,0.110195
36,ps_reg_02,0.100473
42,ps_car_15,0.089937
0,ps_ind_06_bin,0.088220
22,ps_car_03_cat,0.081255
1,ps_ind_07_bin,0.080559
23,ps_car_04_cat,0.072261
35,ps_reg_01,0.069329


In [6]:




submission = pd.read_csv('output/sample_submission.csv')

submission_output = submission

submission_output['target'] = xgb_pred


submission_output.to_csv('output/submission_output_xgb.csv',index=False)

print ("File generated!")


File generated!
